In [ ]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder


In [ ]:
# Load dataset
df = pd.read_csv("../data/bank-additional-full.csv", sep=";")

# Quick inspection
print("Shape:", df.shape)
print("\nTarget distribution:")
print(df["y"].value_counts())


In [ ]:
# Encode target variable: yes -> 1, no -> 0
df["y"] = df["y"].map({"yes": 1, "no": 0})


In [ ]:
# separate features and target
X = df.drop("y", axis=1)
y = df["y"]


In [ ]:
# identify categorical and numerical features
categorical_features = X.select_dtypes(include=["object"]).columns.tolist()
numerical_features = X.select_dtypes(include=["int64", "float64"]).columns.tolist()

print("Categorical Features:", categorical_features)
print("Numerical Features:", numerical_features)


In [ ]:

#prerocessing pipelines for both numeric and categorical data
# Decisions applied here:

# One-Hot Encoding for categorical features

# Standard Scaling for numerical features

# "unknown" treated as a valid category

# drop='first' to avoid multicollinearity


numeric_transformer = StandardScaler()

categorical_transformer = OneHotEncoder(
    drop="first",
    handle_unknown="ignore"
)

preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, numerical_features),
        ("cat", categorical_transformer, categorical_features)
    ]
)


In [ ]:
# split data into train and test sets

X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.2,
    random_state=42,
    stratify=y
)

print("Train shape:", X_train.shape)
print("Test shape:", X_test.shape)


In [ ]:
# applying preprocessing pipelines to train and test data

X_train_processed = preprocessor.fit_transform(X_train)
X_test_processed = preprocessor.transform(X_test)

print("Processed train shape:", X_train_processed.shape)
print("Processed test shape:", X_test_processed.shape)


Preprocessing completed, goal of the above preprocessing was to get

X_train_processed

X_test_processed

y_train

y_test

preprocessor (to be reused in Streamlit)


Moving to model implementation

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
# from xgboost import XGBClassifier

import joblib
import os


In [ ]:
# creating directory to save models


models = {
    "Logistic Regression": LogisticRegression(max_iter=1000, random_state=42),
    
    "Decision Tree": DecisionTreeClassifier(random_state=42),
    
    "kNN": KNeighborsClassifier(n_neighbors=5),
    
    "Naive Bayes": GaussianNB(),
    
    "Random Forest": RandomForestClassifier(
        n_estimators=100,
        random_state=42
    ),
    
    # "XGBoost": XGBClassifier(
    #     use_label_encoder=False,
    #     eval_metric="logloss",
    #     random_state=42
    # )
}


In [ ]:
# Train models

trained_models = {}

for name, model in models.items():
    print(f"Training {name}...")
    model.fit(X_train_processed, y_train)
    trained_models[name] = model

print("All models trained successfully.")


In [ ]:
# save model files


os.makedirs("model", exist_ok=True)

for name, model in trained_models.items():
    filename = name.lower().replace(" ", "_") + ".pkl"
    joblib.dump(model, f"model/{filename}")

print("All models saved successfully.")


Models trained now
Going ahead with evaluation metrics

In [ ]:
from sklearn.metrics import (
    accuracy_score,
    roc_auc_score,
    precision_score,
    recall_score,
    f1_score,
    matthews_corrcoef,
    confusion_matrix,
    classification_report
)

import pandas as pd


In [ ]:
def evaluate_model(model, X_test, y_test):
    y_pred = model.predict(X_test)
    
    # Some models need predict_proba for AUC
    if hasattr(model, "predict_proba"):
        y_prob = model.predict_proba(X_test)[:, 1]
    else:
        y_prob = None

    accuracy = accuracy_score(y_test, y_pred)
    auc = roc_auc_score(y_test, y_prob) if y_prob is not None else None
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    mcc = matthews_corrcoef(y_test, y_pred)

    return accuracy, auc, precision, recall, f1, mcc


In [ ]:
results = []

for name, model in trained_models.items():
    accuracy, auc, precision, recall, f1, mcc = evaluate_model(
        model,
        X_test_processed,
        y_test
    )
    
    results.append([
        name,
        accuracy,
        auc,
        precision,
        recall,
        f1,
        mcc
    ])


In [ ]:
comparison_df = pd.DataFrame(
    results,
    columns=[
        "ML Model Name",
        "Accuracy",
        "AUC",
        "Precision",
        "Recall",
        "F1 Score",
        "MCC"
    ]
)

print(comparison_df)


In [ ]:
for name, model in trained_models.items():
    y_pred = model.predict(X_test_processed)
    cm = confusion_matrix(y_test, y_pred)
    
    print(f"\nConfusion Matrix for {name}:")
    print(cm)
